In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import os
import seaborn as sns
import statsmodels.formula.api as sm
from numpy.polynomial.polynomial import polyfit
from sklearn.model_selection import train_test_split
import sklearn
from math import sqrt
import glob

In [22]:
##PC:
path = r'C:/Users/arj26323/Documents/Data/Biomass datasets/Virginia/Yearly point extraction data' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  ## more options can be specified also
#     print(df)

# df = df[df['Species_Code'] == 'A1'] ##eventually, you will want to do this

ntimes = 16 # number of times you repeat each item
yourlist = [2000,2001,2002,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]
df['Year'] = [i for i in yourlist for _ in range(ntimes)]

df

# print(df.iloc[100,])

# for col in df.columns:
#     print(col)

,B1,B2,B3,B4,B5,B6,B7,system:index,site_id,B10,B11,Year
0,0.0310,0.0466,0.0445,0.1370,0.0632,0.2933,0.0331,0,Assateague,NaN,NaN,2000
1,0.0413,0.0568,0.0450,0.0431,0.0173,0.2929,0.0085,1,Bellvue,NaN,NaN,2000
2,0.0321,0.0662,0.0636,0.2729,0.1777,0.2942,0.0779,2,Box_Tree,NaN,NaN,2000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Channel_Point,NaN,NaN,2000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Cushmans,NaN,NaN,2000
...,...,...,...,...,...,...,...,...,...,...,...,...
267,0.0202,0.0249,0.0438,0.0336,0.2108,0.1042,0.0466,11,N_Brownsville,0.2966,0.2939,2017
268,0.0239,0.0289,0.0440,0.0375,0.1725,0.1126,0.0549,12,Oyster,0.2965,0.2938,2017
269,0.0374,0.0445,0.0553,0.0525,0.0877,0.0682,0.0405,13,Steelmans,0.2972,0.2941,2017
270,0.0315,0.0355,0.0467,0.0386,0.1012,0.0683,0.0360,14,Wallops,0.2961,0.2934,2017


In [23]:
##Bands and indices

df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8') ##make sure no other sensors are being used

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 5', (df['B4']-df['B3'])/(df['B4']+df['B3']), \
                      (df['B5']-df['B4'])/(df['B5']+df['B4'])) ##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B1'], df['B2'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B2'], df['B3'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B3'], df['B4'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B4'], df['B5'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B5'], df['B6'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B7'], df['B7'])

##IMPORTANT: again, these bands between landsat 5 and 8 may not be able to be compared directly

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

df

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(df)

,B1,B2,B3,B4,B5,B6,B7,system:index,site_id,B10,...,Red_band,NIR_band,SWIR1_band,SWIR2_band,savi,wdrvi5,nd_r_g,nd_g_b,nd_swir2_nir,nd_swir2_r
0,0.0310,0.0466,0.0445,0.1370,0.0632,0.2933,0.0331,0,Assateague,NaN,...,0.0445,0.1370,0.0632,0.0331,0.203595,0.212389,-0.023052,0.201031,-0.610817,-0.146907
1,0.0413,0.0568,0.0450,0.0431,0.0173,0.2929,0.0085,1,Bellvue,NaN,...,0.0450,0.0431,0.0173,0.0085,-0.004846,-0.352367,-0.115914,0.158002,-0.670543,-0.682243
2,0.0321,0.0662,0.0636,0.2729,0.1777,0.2942,0.0779,2,Box_Tree,NaN,...,0.0636,0.2729,0.1777,0.0779,0.375314,0.364159,-0.020031,0.346897,-0.555872,0.101060
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Channel_Point,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Cushmans,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,0.0202,0.0249,0.0438,0.0336,0.2108,0.1042,0.0466,11,N_Brownsville,0.2966,...,0.0336,0.2108,0.1042,0.0466,0.357066,0.516547,-0.131783,0.275109,-0.637918,0.162095
268,0.0239,0.0289,0.0440,0.0375,0.1725,0.1126,0.0549,12,Oyster,0.2965,...,0.0375,0.1725,0.1126,0.0549,0.285211,0.393939,-0.079755,0.207133,-0.517150,0.188312
269,0.0374,0.0445,0.0553,0.0525,0.0877,0.0682,0.0405,13,Steelmans,0.2972,...,0.0525,0.0877,0.0682,0.0405,0.082474,-0.089777,-0.025974,0.108216,-0.368175,-0.129032
270,0.0315,0.0355,0.0467,0.0386,0.1012,0.0683,0.0360,14,Wallops,0.2961,...,0.0386,0.1012,0.0683,0.0360,0.146765,0.134529,-0.094959,0.136253,-0.475219,-0.034853
